已经预先将1515模型的reaction信息进行过整理
存储在excel的第三个sheet中
sheet中包含了现有的5914个反应的信息
通过gene_GO文件对缺失kcat的反应进行填补

In [2]:
import pandas as pd
import numpy as np
import re
import math

In [3]:
folder = "C:/User Files/media/文件/"
excel = pd.ExcelFile(folder + "gene_GO.xlsx")    # 将excel文件转换为excel对象
dict_df = pd.read_excel(excel, sheet_name=excel.sheet_names, index_col=0) 


In [4]:
df_1 = dict_df[excel.sheet_names[0]]
df_2 = dict_df[excel.sheet_names[1]]
df_3 = dict_df[excel.sheet_names[2]]

In [5]:
%%time
# 利用GO来填补kcat
df = df_3.copy()
for m, row in df_3.iterrows():
    if not math.isnan(row[2]):
        df.at[m, "kcat_final"] = row[2]
    elif math.isnan(row[2]) and not math.isnan(row[1]):
        if not re.search("and", row[0]):
            
            # 对于grp为单个基因的进行填补
            if row[0] in df_1.index:
                GO = df_1.at[row[0], "GO"]    # 获取基因的GO值，类型为array
                
        else:
            
            # 对grp关系为and的进行填补
            gpr = row[0].split(" and ")
            GO = []
            for i in gpr:
                if i in df_1.index:
                    go = df_1.at[i, "GO"]
                try:
                    GO.extend(go)
                except:
                    GO.append(go)
            
        
        level = {}    # 用于存储一个基因所有{GO: level}的字典(只储存有kcat的)
        for i in GO:
            if i in df_2.index:
                level_gene = df_2.at[i, "level"]    # level_gene可能为数字，也可能为array
                try:
                    level[i] = level_gene[0]
                except:
                    level[i] = level_gene
            pass
        
        if level:    # 可能有的基因只有GO值但没有level
            
            level_max = max(level.values())    # 获取前面字典里面最大的value（最大的level），可能有多个key具有相同的level

            level_new = {}    # 储存最大的level的组合
            for k, v in level.items():
                if v == level_max:
                    level_new[k] = v
                pass

            kcat = []    # 用于储存所有的kcat值
            for k, v in level_new.items():
                try:
                    kcat.extend(df_2.at[k, "kcat"])
                except:
                    kcat.append(df_2.at[k, "kcat"])

                pass
            df.at[m, "kcat_final"] = np.median(kcat)*3600
            df.at[m, "correct"] = "GO"
            pass
        
    pass


Wall time: 9.78 s


In [6]:
df.head()

,gpr,mw,kcat,kcat_final,correct
AADDGT,b4481,40.6396,49389.28890,49389.28890,NaN
GALCTND,b4478,42.5229,75632.85924,75632.85924,NaN
DDPGALA,b4477,21.3906,144563.54000,144563.54000,NaN
FRULYSE,b4474,249.3506,80324.38012,80324.38012,NaN
GLYCTO2,b4468 and b2979 and b4467,137.2828,42109.98399,42109.98399,NaN


In [10]:
# 对于没有GO的进行中位数映射
kcat = [i for i in df["kcat_final"] if not math.isnan(i)]
kcat = np.median(kcat)    # 似乎可以直接用df["kcat_final"].median()来替换
for m, row in df.iterrows():
    if math.isnan(row[3]) and not math.isnan(row[1]):
        df.at[m, "kcat_final"] = kcat
        df.at[m, "correct"] = "median"
        
    pass


In [8]:
# df.to_csv(folder + "kcat_fix.csv", sep=",", header=True, index=True)    # 写为csv，字段之间用’, ’分隔

In [18]:
# 对GO填补的反应进行统计
cc = ccc = cccc = 0
for m, row in df.iterrows():
    if row[-1] == "GO":
        cc+=1    # 共填补个数
        
        if re.search("and", row[0]):
            ccc+=1    # grp有and
        else:
            cccc+=1    # gpr无and
    pass

print(cc, ccc, cccc)

778 130 648


In [18]:
df.head()

,gpr,mw,kcat,kcat_final,correct
AADDGT,b4481,40.6396,49389.28890,49389.28890,NaN
GALCTND,b4478,42.5229,75632.85924,75632.85924,NaN
DDPGALA,b4477,21.3906,144563.54000,144563.54000,NaN
FRULYSE,b4474,249.3506,80324.38012,80324.38012,NaN
GLYCTO2,b4468 and b2979 and b4467,137.2828,42109.98399,42109.98399,NaN


In [14]:
# 删除没有没有蛋白的行，并补充kcat/mw

print(len(df.index))
df = df.drop(df[np.isnan(df["kcat_final"])].index)
print(len(df.index))

try:
    df.insert(4, "kcat/mw", df["kcat_final"] / df["mw"],allow_duplicates=0)    # 插入series,修改原数据，长的series会自动对齐
except:
    pass

# 写为新文件作为过程文件
# df.to_csv(folder + "xxx.csv", sep=",", header=True, index=True)    

df

5345
5345


,gpr,mw,kcat,kcat_final,kcat/mw,correct
AADDGT,b4481,40.6396,4.938929e+04,4.938929e+04,1215.299582,NaN
GALCTND,b4478,42.5229,7.563286e+04,7.563286e+04,1778.638316,NaN
DDPGALA,b4477,21.3906,1.445635e+05,1.445635e+05,6758.274195,NaN
FRULYSE,b4474,249.3506,8.032438e+04,8.032438e+04,322.134297,NaN
GLYCTO2,b4468 and b2979 and b4467,137.2828,4.210998e+04,4.210998e+04,306.738965,NaN
GLYCTO3,b4468 and b2979 and b4467,137.2828,3.566077e+04,3.566077e+04,259.761397,NaN
GLYCTO4,b4468 and b2979 and b4467,137.2828,3.992216e+04,3.992216e+04,290.802324,NaN
GUAtpp_no1,b4464,46.7580,NaN,8.640000e+04,1847.812139,GO
GUAtpp_no2,b4064,45.7120,NaN,5.248533e+04,1148.173963,median
NTP12,b4394,36.4253,1.134173e+05,1.134173e+05,3113.696807,NaN
